In [14]:
%serialconnect

Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [15]:
import network

si = network.WLAN(network.AP_IF)
macaddress = "".join("{:02x}".format(x)  for x in si.config("mac"))
print(macaddress)
si.active(True)


30aea44cb0b5


In [17]:
si.config(essid='ESP32W')
si.config(authmode=3, password='123456789')

In [19]:
help(si)

object <WLAN> is of type WLAN
  active -- <function>
  connect -- <function>
  disconnect -- <function>
  status -- <function>
  scan -- <function>
  isconnected -- <function>
  config -- <function>
  ifconfig -- <function>


In [21]:
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
us = ("192.168.4.2", 4353)

In [38]:
s.sendto(b"hi there\n", us)

In [47]:
math.sqrt

In [49]:
import math, time

tstamp0 = 365*24*3600*18+100
lat0, lng0 = 53.4121628,-2.9864259
lat0, lng0 = 53.413522,-1.9062517
velknots, veldir = 0, 0

earthrad = 6378137
nyfac = 2*math.pi*earthrad/360
exfac = nyfac*math.cos(math.radians(lat0))

def tform(dtseconds):
    tstamp = time.localtime(tstamp0 + dtseconds)
    return "%02u%02u%02u.%03d" % \
        (tstamp[3], tstamp[4], tstamp[5], 0)
    
def latlngform(x, y):
    lat = lat0 + y/nyfac
    lng = lng0 + x/exfac
    return "%02d%06.3f,%c,%02d%06.3f,%c" % \
        (int(abs(lat)), (abs(lat)-math.floor(abs(lat)))*60, 'N' if lat>0 else 'S', 
         int(abs(lng)), (abs(lng)-math.floor(abs(lng)))*60, 'E' if lng>0 else 'W')

def wrapchecksum(rec):
    s = 0
    for c in rec:
        s ^= ord(c)
    return "${:s}*{:02x}\r\n".format(rec, s).encode()

prevx, prevy, prevdtseconds = 0, 0, 0
def gprmcrec(dtseconds, x, y):
    global prevx, prevy, prevdtseconds
    dx, dy, ddt = x-prevx, y-prevy, dtseconds-prevdtseconds
    velknots = math.sqrt(dx**2 + dy**2)*1.94384/max(0.1, ddt)
    veldir = (360+math.degrees(math.atan2(dx, dy)))%360
    prevx, prevy, prevdtseconds = x, y, dtseconds
    #velknots, veldir = 0, 0
    tstamp = time.localtime(tstamp0 + dtseconds)
    rec = "GPRMC,%s,A,%s,%05.1f,%05.1f,%02u%02u%02u,000.0," % \
        (tform(dtseconds), latlngform(x, y), 
         min(velknots, 100), veldir, tstamp[2], tstamp[1], tstamp[0]%100)
    return wrapchecksum(rec)

def gpggarec(dtseconds, x, y, alt):
    rec = "GPGGA,%s,%s,1,12,1.0,%.1f,M,0.0,M,," % \
        (tform(dtseconds), latlngform(x, y), alt)
    return wrapchecksum(rec)

#$GPRMC,153412.410,A,5230.600,N,01323.246,E,2027.8,289.2,310718,000.0,W*4C
#$GPGGA,153413.410,5230.879,N,01322.443,E,1,12,1.0,0.0,M,0.0,M,,*69
gprmcrec(1, 100, 0)
print(gpggarec(1, 100, 0, 100))


b'$GPGGA,000141.000,5324.811,N,0154.285,W,1,12,1.0,100.0,M,0.0,M,,*4e\r\n'


In [54]:
import time, math
circlerad = 90
circleperiod = 40   
climbrate = 1.2
tperiod = 1

def ust(rec):
    s.sendto(rec, us)

for i in range(4000):
    t = int(i/tperiod)
    theta = math.radians(360*t/circleperiod)
    x, y = math.sin(theta)*circlerad, math.cos(theta)*circlerad
    alt = 800+x
    x = t*circlerad/10
    #alt = 800+t*climbrate
    ust(gprmcrec(t, x, y))
    ust(gpggarec(t, x, y, alt))
    time.sleep(1/tperiod)
    

....................................................................
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

